In [1]:
from mdagent import MDAgent

In [2]:
#todays date and time
import datetime

start = datetime.datetime.now()
date = start.strftime("%Y-%m-%d")
print("date:",date)
time = start.strftime("%H:%M:%S")
print("time:",time)

date: 2024-09-18
time: 15:48:06


In [3]:
descriptive_prompt_4 = "Complete the following MD tasks: 1. Download the PDB file 1ZNI. 2. Pre-process the file by adding missing atoms and replacing non-standard residues. 3. Simulate for 1 ps at 300 K. 4. Calculate the RMSD of the simulation."
non_descriptive_prompt_4 = "Simulate protein 1ZNI at 300 K for 1 ps and calculate the RMSD."

In [4]:
llm_model = "gpt-3.5-turbo-0125"
tools = "all"

In [5]:
agent_1 = MDAgent(agent_type="Structured", model=llm_model, top_k_tools=tools)

##### descriptive prompt

In [6]:
start1 = datetime.datetime.now()
agent_1.run(descriptive_prompt_4)

Thought: The task involves multiple steps in molecular dynamics, including downloading a specific PDB file, pre-processing it, simulating it, and calculating the RMSD. Each step requires a specific tool to accomplish.

Action:
```
{
    "action": "PDBFileDownloader",
    "action_input": {
        "query": "1ZNI"
    }
}
```PDB file found with this ID: 1ZNI
Thought: The PDB file 1ZNI has been successfully downloaded. The next step is to pre-process the file by adding missing atoms and replacing non-standard residues.

Action:
```
{
    "action": "CleaningToolFunction",
    "action_input": {
        "pdb_id": "1ZNI_154809",
        "replace_nonstandard_residues": true,
        "add_missing_atoms": true,
        "remove_heterogens": true,
        "remove_water": true,
        "add_hydrogens": true,
        "add_hydrogens_ph": 7.0
    }
}
```file path: /Users/jorgemedina/Desktop/md-agent/ckpt/ckpt_373/pdb/1ZNI_raw_20240918_154809.pdb
Thought: The PDB file 1ZNI has been successfully pre-pro

({'input': '\n    You are an expert molecular dynamics scientist, and\n    your task is to respond to the question or\n    solve the problem to the best of your ability using\n    the provided tools.\n\n    You can only respond with a single complete\n    \'Thought, Action, Action Input\' format\n    OR a single \'Final Answer\' format.\n\n    Complete format:\n    Thought: (reflect on your progress and decide what to do next)\n    Action:\n    ```\n    {\n        "action": (the action name, it should be the name of a tool),\n        "action_input": (the input string for the action)\n    }\n    \'\'\'\n\n    OR\n\n    Final Answer: (the final response to the original input\n    question, once all steps are complete)\n\n    You are required to use the tools provided,\n    using the most specific tool\n    available for each action.\n    Your final answer should contain all information\n    necessary to answer the question and its subquestions.\n    Before you finish, reflect on your pro

In [7]:
end = datetime.datetime.now()
elapsed = end - start1
print(f"agent_1 duration: {elapsed.total_seconds()/60:.2f} minutes")

agent_1 duration: 0.82 minutes


In [8]:
agent_1.path_registry.list_path_names()

'Names found in registry: 1ZNI_154809, 1ZNI_154813, top_sim0_154821, sim0_154821, rec0_154821, rec1_154821, rec2_154821'

In [ ]:
registry = agent_1.path_registry
all_names = registry.list_path_names()
print(all_names)

In [ ]:
paths_and_descriptions = registry.list_path_names_and_descriptions()
print("\n".join(paths_and_descriptions.split(",")))

In [11]:
# let's see if we can't grab the plot
import re
import os
match = re.search(rf"fig0_\d+", all_names)
plot_path = registry.get_mapped_path(match.group(0))
assert os.path.exists(plot_path)

In [ ]:
from IPython.display import Image
Image(filename=plot_path)

##### non-descriptive prompt

In [9]:
agent_2 = MDAgent(agent_type="Structured", model=llm_model, top_k_tools=tools)
start2 = datetime.datetime.now()
agent_2.run(non_descriptive_prompt_4)

Thought: To simulate protein 1ZNI at 300 K for 1 ps and calculate the RMSD, I will need to set up and run a short simulation using the provided parameters.

Action:
```
{
    "action": "SetUpandRunFunction",
    "action_input": {
        "pdb_id": "1ZNI",
        "forcefield_files": ["amber14/protein.ff14SB.xml", "amber14/tip3p.xml"],
        "save": true,
        "system_params": {
            "nonbondedMethod": "NoCutoff",
            "constraints": "HBonds",
            "rigidWater": true
        },
        "integrator_params": {
            "integrator_type": "LangevinMiddle",
            "Temperature": "300 * kelvin",
            "Friction": "1.0 / picoseconds",
            "Timestep": "0.002 * picoseconds"
        },
        "simulation_params": {
            "Ensemble": "NVT",
            "Number of Steps": 500000,
            "record_interval_steps": 100,
            "record_params": ["step", "potentialEnergy", "temperature"]
        }
    }
}
```error found: constraintToleranc

({'input': '\n    You are an expert molecular dynamics scientist, and\n    your task is to respond to the question or\n    solve the problem to the best of your ability using\n    the provided tools.\n\n    You can only respond with a single complete\n    \'Thought, Action, Action Input\' format\n    OR a single \'Final Answer\' format.\n\n    Complete format:\n    Thought: (reflect on your progress and decide what to do next)\n    Action:\n    ```\n    {\n        "action": (the action name, it should be the name of a tool),\n        "action_input": (the input string for the action)\n    }\n    \'\'\'\n\n    OR\n\n    Final Answer: (the final response to the original input\n    question, once all steps are complete)\n\n    You are required to use the tools provided,\n    using the most specific tool\n    available for each action.\n    Your final answer should contain all information\n    necessary to answer the question and its subquestions.\n    Before you finish, reflect on your pro

In [10]:
end = datetime.datetime.now()
elapsed = end - start2
print(f"agent_2 duration: {elapsed.total_seconds()/60:.2f} minutes")

agent_2 duration: 0.35 minutes


In [11]:
registry = agent_2.path_registry
all_names = registry.list_path_names()
print(all_names)

No names found. The JSON file is empty or does not contain name mappings.


In [12]:
paths_and_descriptions = registry.list_path_names_and_descriptions()
print("\n".join(paths_and_descriptions.split(",")))

No names found. The JSON file is empty or does not contain name mappings.


In [17]:
# let's see if we can't grab the plot
import re
import os
match = re.search(rf"fig0_\d+", all_names)
plot_path = registry.get_mapped_path(match.group(0))
assert os.path.exists(plot_path)

AttributeError: 'NoneType' object has no attribute 'group'

In [ ]:
from IPython.display import Image
Image(filename=plot_path)